In [1]:
from utils import get_data
import numpy
import pandas as pd

In [2]:
data = get_data('../data')

In [3]:
# Find home that have been sold
# removing condos since they the units may have the same address, and trying to match on address.
soldHomes = data[data['STATUS'] == 'SLD']
soldHomes = soldHomes[soldHomes['PROPTYPE'] != 'CC']

In [4]:
soldMLS = soldHomes['MLSNUM'].unique()

In [5]:
# find home with same address
def getHome(data, mls):
    home = soldHomes[soldHomes['MLSNUM'] == mls]
    homes = soldHomes[soldHomes['ZIP'] == home['ZIP'].iloc[0]]
    homes = homes[homes['HOUSENUM1'] == home['HOUSENUM1'].iloc[0]]
    homes = homes[homes['STREETNAME'] == home['STREETNAME'].iloc[0]]
    return homes

In [ ]:
# go through all mls listing to find matching homes.
test_homes = []
for mls in soldMLS:
    homes = getHome(soldHomes, mls)
    if(homes.shape[0] == 2):
        test_homes.append(homes)

In [ ]:
flippedHomes = pd.concat(test_homes)

In [ ]:
# remove duplicates and sort by mls
flippedHomes = flippedHomes.drop_duplicates(subset='MLSNUM', keep="first")
flippedHomes = flippedHomes.sort_values(by='MLSNUM')

In [ ]:
# build flip set
flips = [];
for mls in flippedHomes['MLSNUM']:
    homes = getHome(flippedHomes, mls)
    # check if home has 2 mls rows
    if(homes.shape[0] >= 2):
        pre = homes.iloc[0]
        post = homes.iloc[1]
        # check if it's not the same mls number
        if pre['MLSNUM'] == post['MLSNUM']:
            continue
        # find pre and post flip values
        if pd.to_datetime(pre['SOLDDATE']) > pd.to_datetime(post['SOLDDATE']):
            # post < pre: swap them
            swap = pre
            pre = post
            post = swap
        flips.append({
            'PRE-FLIP-MLS': pre['MLSNUM'],
            'POST-FLIP-MLS': post['MLSNUM'],
            'PRE-FLIP-SOLD-DATE': pre['SOLDDATE'],
            'POST-FLIP-SOLD-DATE': post['SOLDDATE'],
            'PRE-SOLD-PRICE': pre['SOLDPRICE'],
            'POST-SOLD-PRICE': post['SOLDPRICE'],
            'PRE-SOLD-PRICE': pre['SOLDPRICE'],
            'POST-SOLD-PRICE': post['SOLDPRICE'],
            'PROFIT': int(post['SOLDPRICE']) - int(pre['SOLDPRICE'])
        })

In [ ]:
flips = pd.DataFrame.from_dict(flips)

In [ ]:
flips.to_csv('./outputs/flips.csv')

In [ ]:
flips.head()